<a href="https://colab.research.google.com/github/fleckc/visSP23/blob/main/ps2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import urllib
import os
from google.colab import files
import time, os, sys, re
import pandas_datareader as pdr
from pandas_datareader import wb

Looking at NJ population, voting habits, and migration. This uses election turnout results from 2000, the 2000 census population counts, and the 2000 census migration patterns in order to retain year similarity.

aok ok good but again dont forget to give url to the source data so i can find the raw data online easily myself

In [4]:
urllib.request.urlretrieve("https://drive.google.com/file/d/1Zb6_D7M_KW4juFJpo9XQBlNV6FDg2NIK/view?usp=sharing", "NJ2000ballots.csv")
#aok it breaks; 2 problems here: need to make it public and need to make the proper link as explained in
#https://theaok.github.io/generic/howToPutDataOnline.html

('NJ2000ballots.csv', <http.client.HTTPMessage at 0x7fd804eb14c0>)

In [5]:
njvote=pd.read_csv('/content/NJ2000ballots.csv')

ParserError: ignored

In [ ]:
urllib.request.urlretrieve("https://drive.google.com/file/d/1RhnuEtZrB7kT-DqLbWlgO8e4Aj1dCP2q/view?usp=share_link", "njpop2000.csv")

('njpop2000.csv', <http.client.HTTPMessage at 0x7f757ddbc850>)

In [ ]:
njpop=pd.read_csv('/content/njpop2000.csv')

In [ ]:
urllib.request.urlretrieve("https://drive.google.com/file/d/1n0CQ8bgKBN2-JYOgBszEh_G6QWJK-QI-/view?usp=share_link", "countymigrationinflow.txt")

('countymigrationinflow.txt', <http.client.HTTPMessage at 0x7f757ddbc2e0>)

In [ ]:
njmigrate=pd.read_fwf('/content/countymigrationinflow.txt')

In [ ]:
njvote=njvote.rename(columns={'COUNTY': 'County'})
njvote #renaming for easier merge later

In [ ]:
njpop.head()

In [ ]:
countiesd = {'County': ['Atlantic', 'Bergen', 'Burlington', 'Camden', 'Cape May', 'Cumberland', 'Essex', 'Gloucester', 'Hudson', 'Hunterdon', 'Mercer', 'Middlesex', 'Monmouth', 'Morris', 'Ocean', 'Passaic', 'Salem', 'Somerset', 'Sussex', 'Union', 'Warren']}
counties = pd.DataFrame(countiesd)
counties #creating a dataframe (from a dict) for the list of counties to "fix" unmatched county name formats (Atlantic vs Atlantic County)

In [ ]:
njpop=njpop.drop(njpop.index[21:31])
njpop=njpop.drop(['County'], axis=1)
njpop=counties.join(njpop, how="outer")
njpop #dropping extraneous info and renaming counties column for easier merge later

In [ ]:
njmigrate.head()

In [ ]:
njmigrate=njmigrate.rename(columns={'01001': 'StateCounty2000',
                 '01003': 'StateCounty1995',
                 '87': 'InflowMigration'})
#renaming columns - note will eliminate some data in this instance but none we will use

In [ ]:
njmigratenjonly=njmigrate.query('StateCounty2000 > 34000 & StateCounty2000 < 35000')
#eliminating FIPS codes/counties that are not NJ

In [ ]:
njmigratenjonly.head()

In [ ]:
njmigrateonly2=njmigratenjonly.groupby('StateCounty2000')['InflowMigration'].sum()
njmigrateonly2=pd.DataFrame(data=njmigrateonly2)
njmigrateonly2=njmigrateonly2.reset_index(drop="true")
#not really what I wanted but it'll work in the end

In [ ]:
njmigrateonly2.head()

In [ ]:
njmerge1 = counties.join(njmigrateonly2, how="outer")
#putting county names to the total migration inflows

In [ ]:
njmerge1.head()

In [ ]:
njmerge2=pd.merge(njmerge1, njvote, how="outer" )
#merging migration and turnout

In [ ]:
njmerge2.head()

In [ ]:
njmerge3= pd.merge(njpop, njmerge2, how="outer", on='County')
#merging everything together into one dataset with common county naming convention

In [ ]:
njmerge3